# Goal and organisation


* I) Download the model
* II) Example to score bias with English model
* III) Example to manipulate the word vectors
* IV) Example with French and Italian. !!! The score does not work because probably the translation are wrong
* V) if usefull, code to doanlowd multiple models at once

In [1]:
# Imports
from pathlib import Path
from gensim.models.fasttext import load_facebook_vectors
from wefe.word_embedding_model import WordEmbeddingModel
from wefe.datasets import load_weat
from wefe.metrics import WEAT
from wefe.query import Query
import numpy as np
from translate import Translator
import re
import pandas as pd

# Download fasttext model 

* Download model from here: https://fasttext.cc/docs/en/crawl-vectors.html
The ".bin" files of the selected languages.
* put the file in a folder of your choice
* update the path below, with the location of the folder where you put the models

e.g. here, in the `socio_wordvec/notebooks` folder, there is the file `cc.en.300.bin`

In [2]:
path_to_fasttext_models = Path("/private/home/ccaucheteux/socio_wordvec/notebooks/")

# Example to score English words

## Load fasttext model

In [3]:
lang="en" # Select the language here, english. 
model_name = f"cc.{lang}.300.bin" # the model file (should be the same, ending by '.bin')
model_path = path_to_fasttext_models / model_name

assert Path(model_path).is_file(), f"{model_path} does not exist"

In [4]:
%%time
fasttext_model = load_facebook_vectors(model_path)

CPU times: user 1min 3s, sys: 4.47 s, total: 1min 8s
Wall time: 1min 8s


In [5]:
# Convert model to be compatible with WEFE library
wefe_model = WordEmbeddingModel(fasttext_model, f"Fasttext model for {lang}")

## Setup the queries and words

In [6]:
gender_query = Query(
    target_sets=[
        ["female", "woman", "girl", "sister", "she", "her", "hers", "daughter"],
        ["male", "man", "boy", "brother", "he", "him", "his", "son"],
    ],
    attribute_sets=[
        [
            "home",
            "parents",
            "children",
            "family",
            "cousins",
            "marriage",
            "wedding",
            "relatives",
        ],
        [
            "executive",
            "management",
            "professional",
            "corporation",
            "salary",
            "office",
            "business",
            "career",
        ],
    ],
    target_sets_names=["Female terms", "Male Terms"],
    attribute_sets_names=["Family", "Careers"],
)

gender_query

<Query: Female terms and Male Terms wrt Family and Careers
- Target sets: [['female', 'woman', 'girl', 'sister', 'she', 'her', 'hers', 'daughter'], ['male', 'man', 'boy', 'brother', 'he', 'him', 'his', 'son']]
- Attribute sets:[['home', 'parents', 'children', 'family', 'cousins', 'marriage', 'wedding', 'relatives'], ['executive', 'management', 'professional', 'corporation', 'salary', 'office', 'business', 'career']]>

## Compute score using the WEFE library

In [7]:
metric = WEAT()
result = metric.run_query(gender_query, wefe_model)
result

{'query_name': 'Female terms and Male Terms wrt Family and Careers',
 'result': 0.3361583009827882,
 'weat': 0.3361583009827882,
 'effect_size': 0.398944696302527,
 'p_value': nan}

# Example to extract vectors

In [8]:
fasttext_model["woman"]

array([ 1.32745296e-01, -5.15583158e-02,  1.42445713e-02,  7.71068111e-02,
       -6.10234290e-02, -7.63783678e-02,  5.75574599e-02,  9.24359309e-04,
       -1.66228879e-02,  6.36110082e-02, -4.18582149e-02, -4.14778255e-02,
       -3.99447307e-02, -3.18428576e-02, -6.66241050e-02,  7.76687339e-02,
       -1.33534998e-01,  1.43744145e-03, -3.96703370e-04,  8.06799978e-02,
        5.14491387e-02,  2.70009711e-02, -5.71396109e-03,  3.30801047e-02,
       -3.18728909e-02,  3.02043557e-02, -3.17235589e-02, -3.89806964e-02,
        3.07462383e-02,  1.11801632e-01, -1.67721771e-02,  7.24857226e-02,
        1.03701837e-01, -1.09409671e-02, -4.93282303e-02,  5.19027337e-02,
        3.02081648e-03, -7.44009987e-02,  2.31740195e-02, -5.19711934e-02,
       -3.28774266e-02,  3.90867889e-02, -2.64221337e-02,  2.56395955e-02,
       -7.95582235e-02, -2.66516246e-02,  6.22903183e-02, -3.72034237e-02,
       -4.65604961e-02, -4.17536050e-02,  5.54847568e-02,  2.17969622e-03,
        1.13694780e-02,  

In [9]:
fasttext_model["woman"].shape

(300,)

# Example to generalise to other languages

In [10]:
selected_langs = ["fr", "it"]

## Translate Query

In [11]:
target_sets=[
        ["female", "woman", "girl", "sister", "she", "her", "hers", "daughter"],
        ["male", "man", "boy", "brother", "he", "him", "his", "son"],
    ]
attribute_sets=[
    [
        "home",
        "parents",
        "children",
        "family",
        "cousins",
        "marriage",
        "wedding",
        "relatives",
    ], 
    [
        "executive",
        "management",
        "professional",
        "corporation",
        "salary",
        "office",
        "business",
        "career",
    ]
]
target_sets_names=["Female", "Male"]
attribute_sets_names=["Family", "Careers"]

In [12]:
words = []
categories = []
kinds = []
for kind_sets, kind, names in zip([target_sets, attribute_sets], ["target", "attribute"],
                                 [target_sets_names, attribute_sets_names]):
    for category, word_set in zip(names, kind_sets):
        words.extend(word_set)
        categories.extend([category]*len(word_set))
        kinds.extend([kind]*len(word_set))
df = pd.DataFrame({"english_word": words, "category": categories, "kind": kinds})
df

,english_word,category,kind
0,female,Female,target
1,woman,Female,target
2,girl,Female,target
3,sister,Female,target
4,she,Female,target
5,her,Female,target
6,hers,Female,target
7,daughter,Female,target
8,male,Male,target
9,man,Male,target


In [13]:
for lang in selected_langs: # For each language
    translator = Translator(to_lang=lang) # Load specific translator
    translated_word = []
    for word in df.english_word.values: # For each ensglih word
        translation = translator.translate(word) # Translate

        # Preprocess
        clean_translation = re.sub(r'\W+', '', translation).lower()

        print(f"{lang} translation for word '{word}' : '{clean_translation}'")
        translated_word.append(clean_translation) # Update the list
    df[lang] = translated_word

fr translation for word 'female' : 'desexefeminin'
fr translation for word 'woman' : 'femme'
fr translation for word 'girl' : 'fille'
fr translation for word 'sister' : 'sœur'
fr translation for word 'she' : 'elle'
fr translation for word 'her' : 'sa'
fr translation for word 'hers' : 'sa'
fr translation for word 'daughter' : 'fille'
fr translation for word 'male' : 'masculin'
fr translation for word 'man' : 'homme'
fr translation for word 'boy' : 'garçon'
fr translation for word 'brother' : 'frère'
fr translation for word 'he' : 'il'
fr translation for word 'him' : 'lui'
fr translation for word 'his' : 'son'
fr translation for word 'son' : 'fils'
fr translation for word 'home' : 'accueil'
fr translation for word 'parents' : 'parents'
fr translation for word 'children' : 'enfants'
fr translation for word 'family' : 'famille'
fr translation for word 'cousins' : 'cousins'
fr translation for word 'marriage' : 'lemariage'
fr translation for word 'wedding' : 'mariage'
fr translation for word

In [14]:
df

,english_word,category,kind,fr,it
0,female,Female,target,desexefeminin,femminile
1,woman,Female,target,femme,donna
2,girl,Female,target,fille,telugu
3,sister,Female,target,sœur,sorella
4,she,Female,target,elle,lei
5,her,Female,target,sa,leisuo
6,hers,Female,target,sa,lasua
7,daughter,Female,target,fille,figlia
8,male,Male,target,masculin,uomo
9,man,Male,target,homme,man


In [17]:
def build_query(target_sets, attribute_sets):
    gender_query = Query(
    target_sets=target_sets,
    attribute_sets=attribute_sets,
    target_sets_names=["Female", "Male"],
    attribute_sets_names=["Family", "Careers"]
)
    return gender_query

In [18]:
all_results = {}

for lang in selected_langs:
    
    # Extract translated words to build translated query
    target_sets = [
        df.query("kind=='target' and category=='Female'")[lang].values.astype(str),
        df.query("kind=='target' and category=='Male'")[lang].values.astype(str)
                  ]
    attribute_sets = [
        df.query("kind=='attribute' and category=='Family'")[lang].values.astype(str),
        df.query("kind=='attribute' and category=='Careers'")[lang].values.astype(str)
                     ]
    assert np.array(target_sets).shape == np.array(attribute_sets).shape
    gender_query = build_query(target_sets, attribute_sets)
    print(f"Query for language {lang}: {gender_query}")
    
    # Load model
    print(f"Loading model for language {lang}...")
    model_name = f"cc.{lang}.300.bin"
    model_path = path_to_fasttext_models / model_name
    fasttext_model = load_facebook_vectors(model_path)
    wefe_model = WordEmbeddingModel(fasttext_model, f"Fasttext model for {lang}")
    print(f"Model loaded")
    
    # Compute metric
    print(f"Computing metric for {lang}...")
    metric = WEAT()
    result = metric.run_query(gender_query, wefe_model)
    
    # Print result
    print(f"Result for {lang}: {result}")
    all_results[lang] = result

Query for language fr: <Query: Female and Male wrt Family and Careers
- Target sets: [array(['desexefeminin', 'femme', 'fille', 'sœur', 'elle', 'sa', 'sa',
       'fille'], dtype='<U13'), array(['masculin', 'homme', 'garçon', 'frère', 'il', 'lui', 'son', 'fils'],
      dtype='<U8')]
- Attribute sets:[array(['accueil', 'parents', 'enfants', 'famille', 'cousins', 'lemariage',
       'mariage', 'personnesparentes'], dtype='<U17'), array(['exécutif', 'concernée', 'professionnel', 'compagnie', 'salaire',
       'office', 'business', 'carrières'], dtype='<U13')]>
Loading model for language fr...


ERROR:root:At least one set of 'Female and Male wrt Family and Careers' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.


Model loaded
Computing metric for fr...
Result for fr: {'query_name': 'Female and Male wrt Family and Careers', 'result': nan, 'weat': nan, 'effect_size': nan}
Query for language it: <Query: Female and Male wrt Family and Careers
- Target sets: [array(['femminile', 'donna', 'telugu', 'sorella', 'lei', 'leisuo',
       'lasua', 'figlia'], dtype='<U9'), array(['uomo', 'man', 'ragazzo', 'fratello', 'lui', 'lui', 'sue',
       'figlio'], dtype='<U8')]
- Attribute sets:[array(['home', 'parents', 'bambini', 'famiglia', 'auguri', 'matrimonio',
       'matrimonio', 'parent'], dtype='<U10'), array(['dirigente', 'strutturato', 'professionista', 'societàdicapitali',
       'stipendio', 'ufficio', 'business', 'carriera'], dtype='<U17')]>
Loading model for language it...
Model loaded
Computing metric for it...
Result for it: {'query_name': 'Female and Male wrt Family and Careers', 'result': 0.22531319143516676, 'weat': 0.22531319143516676, 'effect_size': 0.5224221847895621, 'p_value': nan}


# Utils to automatically download models

In [ ]:
import fasttext.util
from tqdm.notebook import tqdm

selected_langs = []

# Extremely slow... >30-60min per language. Yet, if ran locally, just one time is needed. 
models = {}
for lang in selected_langs:
    print(f"Downloading fasttext model for {lang}...")
    fasttext.util.download_model(lang, if_exists='ignore')
    print("Done")

You can find the language ID here (CTRL F on the language): 

In [ ]:
languages = {'Afrikaans' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.af.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.af.300.vec.gz">text</a></td><td>
             'Albanian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sq.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sq.300.vec.gz">text</a></td><td>
             'Alemannic' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.als.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.als.300.vec.gz">text</a></td></tr>
'Amharic' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.am.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.am.300.vec.gz">text</a></td><td>Arabic' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ar.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ar.300.vec.gz">text</a></td><td>Aragonese' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.an.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.an.300.vec.gz">text</a></td></tr>
'Armenian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hy.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hy.300.vec.gz">text</a></td><td>Assamese' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.as.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.as.300.vec.gz">text</a></td><td>Asturian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ast.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ast.300.vec.gz">text</a></td></tr>
'Azerbaijani' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.az.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.az.300.vec.gz">text</a></td><td>Bashkir' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ba.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ba.300.vec.gz">text</a></td><td>Basque' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.eu.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.eu.300.vec.gz">text</a></td></tr>
'Bavarian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bar.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bar.300.vec.gz">text</a></td><td>Belarusian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.be.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.be.300.vec.gz">text</a></td><td>Bengali' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bn.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bn.300.vec.gz">text</a></td></tr>
'Bihari' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bh.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bh.300.vec.gz">text</a></td><td>Bishnupriya Manipuri' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bpy.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bpy.300.vec.gz">text</a></td><td>Bosnian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bs.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bs.300.vec.gz">text</a></td></tr>
'Breton' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.br.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.br.300.vec.gz">text</a></td><td>Bulgarian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bg.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bg.300.vec.gz">text</a></td><td>Burmese' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.my.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.my.300.vec.gz">text</a></td></tr>
'Catalan' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ca.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ca.300.vec.gz">text</a></td><td>Cebuano' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ceb.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ceb.300.vec.gz">text</a></td><td>Central Bicolano' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bcl.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bcl.300.vec.gz">text</a></td></tr>
'Chechen' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ce.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ce.300.vec.gz">text</a></td><td>Chinese' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.zh.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.zh.300.vec.gz">text</a></td><td>Chuvash' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.cv.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.cv.300.vec.gz">text</a></td></tr>
'Corsican' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.co.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.co.300.vec.gz">text</a></td><td>Croatian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hr.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hr.300.vec.gz">text</a></td><td>Czech' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.cs.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.cs.300.vec.gz">text</a></td></tr>
'Danish' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.da.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.da.300.vec.gz">text</a></td><td>Divehi' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.dv.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.dv.300.vec.gz">text</a></td><td>Dutch' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.nl.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.nl.300.vec.gz">text</a></td></tr>
'Eastern Punjabi' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pa.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pa.300.vec.gz">text</a></td><td>Egyptian Arabic' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.arz.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.arz.300.vec.gz">text</a></td><td>Emilian-Romagnol' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.eml.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.eml.300.vec.gz">text</a></td></tr>
'English: <a href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz">text</a></td><td>Erzya' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.myv.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.myv.300.vec.gz">text</a></td><td>Esperanto' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.eo.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.eo.300.vec.gz">text</a></td></tr>
'Estonian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.et.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.et.300.vec.gz">text</a></td><td>Fiji Hindi' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hif.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hif.300.vec.gz">text</a></td><td>Finnish' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fi.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fi.300.vec.gz">text</a></td></tr>
'French' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz">text</a></td><td>Galician' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.gl.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.gl.300.vec.gz">text</a></td><td>Georgian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ka.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ka.300.vec.gz">text</a></td></tr>
'German' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.de.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.de.300.vec.gz">text</a></td><td>Goan Konkani' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.gom.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.gom.300.vec.gz">text</a></td><td>Greek' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.el.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.el.300.vec.gz">text</a></td></tr>
'Gujarati' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.gu.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.gu.300.vec.gz">text</a></td><td>Haitian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ht.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ht.300.vec.gz">text</a></td><td>Hebrew' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.he.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.he.300.vec.gz">text</a></td></tr>
'Hill Mari' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mrj.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mrj.300.vec.gz">text</a></td><td>Hindi' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hi.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hi.300.vec.gz">text</a></td><td>Hungarian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hu.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hu.300.vec.gz">text</a></td></tr>
'Icelandic' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.is.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.is.300.vec.gz">text</a></td><td>Ido' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.io.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.io.300.vec.gz">text</a></td><td>Ilokano' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ilo.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ilo.300.vec.gz">text</a></td></tr>
'Indonesian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.vec.gz">text</a></td><td>Interlingua' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ia.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ia.300.vec.gz">text</a></td><td>Irish' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ga.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ga.300.vec.gz">text</a></td></tr>
'Italian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.it.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.it.300.vec.gz">text</a></td><td>Japanese' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ja.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ja.300.vec.gz">text</a></td><td>Javanese' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.jv.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.jv.300.vec.gz">text</a></td></tr>
'Kannada' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.kn.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.kn.300.vec.gz">text</a></td><td>Kapampangan' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pam.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pam.300.vec.gz">text</a></td><td>Kazakh' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.kk.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.kk.300.vec.gz">text</a></td></tr>
'Khmer' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.km.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.km.300.vec.gz">text</a></td><td>Kirghiz' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ky.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ky.300.vec.gz">text</a></td><td>Korean' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ko.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ko.300.vec.gz">text</a></td></tr>
'Kurdish (Kurmanji)' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ku.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ku.300.vec.gz">text</a></td><td>Kurdish (Sorani)' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ckb.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ckb.300.vec.gz">text</a></td><td>Latin' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.la.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.la.300.vec.gz">text</a></td></tr>
'Latvian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.lv.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.lv.300.vec.gz">text</a></td><td>Limburgish' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.li.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.li.300.vec.gz">text</a></td><td>Lithuanian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.lt.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.lt.300.vec.gz">text</a></td></tr>
'Lombard' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.lmo.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.lmo.300.vec.gz">text</a></td><td>Low Saxon' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.nds.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.nds.300.vec.gz">text</a></td><td>Luxembourgish' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.lb.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.lb.300.vec.gz">text</a></td></tr>
'Macedonian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mk.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mk.300.vec.gz">text</a></td><td>Maithili' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mai.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mai.300.vec.gz">text</a></td><td>Malagasy' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mg.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mg.300.vec.gz">text</a></td></tr>
'Malay' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ms.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ms.300.vec.gz">text</a></td><td>Malayalam' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ml.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ml.300.vec.gz">text</a></td><td>Maltese' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mt.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mt.300.vec.gz">text</a></td></tr>
'Manx' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.gv.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.gv.300.vec.gz">text</a></td><td>Marathi' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mr.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mr.300.vec.gz">text</a></td><td>Mazandarani' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mzn.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mzn.300.vec.gz">text</a></td></tr>
'Meadow Mari' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mhr.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mhr.300.vec.gz">text</a></td><td>Minangkabau' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.min.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.min.300.vec.gz">text</a></td><td>Mingrelian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.xmf.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.xmf.300.vec.gz">text</a></td></tr>
'Mirandese' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mwl.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mwl.300.vec.gz">text</a></td><td>Mongolian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mn.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mn.300.vec.gz">text</a></td><td>Nahuatl' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.nah.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.nah.300.vec.gz">text</a></td></tr>
'Neapolitan' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.nap.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.nap.300.vec.gz">text</a></td><td>Nepali' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ne.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ne.300.vec.gz">text</a></td><td>Newar' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.new.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.new.300.vec.gz">text</a></td></tr>
'North Frisian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.frr.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.frr.300.vec.gz">text</a></td><td>Northern Sotho' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.nso.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.nso.300.vec.gz">text</a></td><td>Norwegian (Bokmål)' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.no.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.no.300.vec.gz">text</a></td></tr>
'Norwegian (Nynorsk)' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.nn.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.nn.300.vec.gz">text</a></td><td>Occitan' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.oc.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.oc.300.vec.gz">text</a></td><td>Oriya' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.or.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.or.300.vec.gz">text</a></td></tr>
'Ossetian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.os.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.os.300.vec.gz">text</a></td><td>Palatinate German' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pfl.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pfl.300.vec.gz">text</a></td><td>Pashto' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ps.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ps.300.vec.gz">text</a></td></tr>
'Persian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.vec.gz">text</a></td><td>Piedmontese' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pms.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pms.300.vec.gz">text</a></td><td>Polish' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pl.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pl.300.vec.gz">text</a></td></tr>
'Portuguese' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pt.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pt.300.vec.gz">text</a></td><td>Quechua' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.qu.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.qu.300.vec.gz">text</a></td><td>Romanian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ro.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ro.300.vec.gz">text</a></td></tr>
'Romansh' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.rm.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.rm.300.vec.gz">text</a></td><td>Russian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz">text</a></td><td>Sakha' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sah.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sah.300.vec.gz">text</a></td></tr>
'Sanskrit' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sa.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sa.300.vec.gz">text</a></td><td>Sardinian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sc.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sc.300.vec.gz">text</a></td><td>Scots' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sco.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sco.300.vec.gz">text</a></td></tr>
'Scottish Gaelic' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.gd.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.gd.300.vec.gz">text</a></td><td>Serbian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sr.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sr.300.vec.gz">text</a></td><td>Serbo-Croatian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sh.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sh.300.vec.gz">text</a></td></tr>
'Sicilian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.scn.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.scn.300.vec.gz">text</a></td><td>Sindhi' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sd.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sd.300.vec.gz">text</a></td><td>Sinhalese' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.si.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.si.300.vec.gz">text</a></td></tr>
'Slovak' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sk.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sk.300.vec.gz">text</a></td><td>Slovenian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sl.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sl.300.vec.gz">text</a></td><td>Somali' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.so.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.so.300.vec.gz">text</a></td></tr>
'Southern Azerbaijani' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.azb.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.azb.300.vec.gz">text</a></td><td>Spanish' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.vec.gz">text</a></td><td>Sundanese' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.su.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.su.300.vec.gz">text</a></td></tr>
'Swahili' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sw.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sw.300.vec.gz">text</a></td><td>Swedish' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sv.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.sv.300.vec.gz">text</a></td><td>Tagalog' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tl.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tl.300.vec.gz">text</a></td></tr>
'Tajik' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tg.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tg.300.vec.gz">text</a></td><td>Tamil' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ta.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ta.300.vec.gz">text</a></td><td>Tatar' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tt.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tt.300.vec.gz">text</a></td></tr>
'Telugu' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.te.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.te.300.vec.gz">text</a></td><td>Thai' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.th.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.th.300.vec.gz">text</a></td><td>Tibetan' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bo.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bo.300.vec.gz">text</a></td></tr>
'Turkish' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.vec.gz">text</a></td><td>Turkmen' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tk.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tk.300.vec.gz">text</a></td><td>Ukrainian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uk.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uk.300.vec.gz">text</a></td></tr>
'Upper Sorbian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hsb.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hsb.300.vec.gz">text</a></td><td>Urdu' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ur.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ur.300.vec.gz">text</a></td><td>Uyghur' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ug.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ug.300.vec.gz">text</a></td></tr>
'Uzbek' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uz.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uz.300.vec.gz">text</a></td><td>Venetian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.vec.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.vec.300.vec.gz">text</a></td><td>Vietnamese' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.vi.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.vi.300.vec.gz">text</a></td></tr>
'Volapük' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.vo.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.vo.300.vec.gz">text</a></td><td>Walloon' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.wa.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.wa.300.vec.gz">text</a></td><td>Waray' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.war.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.war.300.vec.gz">text</a></td></tr>
'Welsh' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.cy.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.cy.300.vec.gz">text</a></td><td>West Flemish' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.vls.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.vls.300.vec.gz">text</a></td><td>West Frisian' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fy.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fy.300.vec.gz">text</a></td></tr>
'Western Punjabi' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pnb.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pnb.300.vec.gz">text</a></td><td>Yiddish' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.yi.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.yi.300.vec.gz">text</a></td><td>Yoruba' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.yo.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.yo.300.vec.gz">text</a></td></tr>
'Zazaki' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.diq.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.diq.300.vec.gz">text</a></td><td>Zeelandic' : "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.zea.300.bin.gz", #href="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.zea.300.vec.gz">text</a></td></tr>
}